In [1]:
from pyspark.sql import Row, SparkSession
spark = SparkSession.builder.appName("movie").getOrCreate()

23/02/19 14:59:07 WARN Utils: Your hostname, DESKTOP-60CN0SA resolves to a loopback address: 127.0.1.1; using 172.29.46.194 instead (on interface eth0)
23/02/19 14:59:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/19 14:59:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/19 14:59:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# RATINGS_FILE = "../data/mllib/sample_movielens_data.txt"
RATINGS_FILE = "ml-latest-small/ratings.csv"
MOVIES_FILE = "ml-latest-small/movies.csv"

In [3]:
import pandas as pd
movies = pd.read_csv(MOVIES_FILE)
movies[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies_df = spark.createDataFrame(movies)
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [5]:
with open(RATINGS_FILE, 'rt') as f:
    ratings = f.readlines()
ratings[:5]

['userId,movieId,rating,timestamp\n',
 '1,1,4.0,964982703\n',
 '1,3,4.0,964981247\n',
 '1,6,4.0,964982224\n',
 '1,47,5.0,964983815\n']

In [6]:
df = spark.read.option('header', 'true').csv(RATINGS_FILE, inferSchema=True)
print(df.count())
display(df.dtypes)
df.show(5)

100836


[('userId', 'int'),
 ('movieId', 'int'),
 ('rating', 'double'),
 ('timestamp', 'int')]

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [7]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=3, userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy='drop')


In [8]:

(train, test) = df.randomSplit([.8, .2])

In [9]:
model = als.fit(train)

23/02/19 14:59:35 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
from pyspark.ml.evaluation import RegressionEvaluator
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction' )

In [11]:
evaluator.evaluate(predictions)

0.8968118009417707

In [12]:
import pyspark.sql.functions as F

In [13]:
model.recommendForAllUsers(1).dtypes

[('userId', 'int'),
 ('recommendations', 'array<struct<movieId:int,rating:float>>')]

In [ ]:
model.recommendForAllUsers(1).select(F.element_at(F.col("recommendations.movieId"),1).alias('movieId')).join(movies_df, on='movieId').show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|    720|Wallace & Gromit:...|Adventure|Animati...|
|    720|Wallace & Gromit:...|Adventure|Animati...|
|    720|Wallace & Gromit:...|Adventure|Animati...|
|   1280|Raise the Red Lan...|               Drama|
|   1241|Dead Alive (Brain...|Comedy|Fantasy|Ho...|
|    932|Affair to Remembe...|       Drama|Romance|
|    932|Affair to Remembe...|       Drama|Romance|
|    417|    Barcelona (1994)|      Comedy|Romance|
|    232|Eat Drink Man Wom...|Comedy|Drama|Romance|
|    232|Eat Drink Man Wom...|Comedy|Drama|Romance|
|    446|Farewell My Concu...|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   Persuasion (1995)|       Drama|Romance|
|     28|   

In [ ]:
model.recommendForAllUsers(1).dtypes

[('userId', 'int'),
 ('recommendations', 'array<struct<movieId:int,rating:float>>')]